In [ ]:
from models.AlexNet import AlexNet
from models.VGG13 import VGG13

from tensorflow.keras.datasets import cifar10
from tensorflow.keras import losses, optimizers
import tensorflow as tf
import numpy as np

### CIFAR-10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
len(x_train), len(x_test)

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# AlexNet

In [ ]:
alexnet = AlexNet((32,32,3), 10)
alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
alexnet.summary()

### Train

In [ ]:
alexnet.fit(x_train, y_train, epochs=100)

### Test

In [ ]:
alexnet.evaluate(x_test, y_test)

# VGG13

In [ ]:
vgg13 = VGG13((32,32, 3), 10)
vgg13.compile(optimizer=optimizers.Adam(learning_rate=1e-2), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg13.summary()

### Train

In [ ]:
vgg13.fit(x_train, y_train, epochs=10)

### Test

In [ ]:
vgg13.evaluate(x_test, y_test)

# FrankNet

### Train

### Test